<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Deep_Learning_Forex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install pandas_ta yfinance

import datetime
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Gather Data
symbol = "EURAUD=X"
start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df_euraud = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Calculate technical indicators
df_euraud["RSI"] = ta.rsi(df_euraud["Close"])
df_euraud[["MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist"]] = ta.macd(df_euraud["Close"])
df_euraud["ATR"] = ta.atr(df_euraud["High"], df_euraud["Low"], df_euraud["Close"])

# Drop NaN values and reset the index
df_euraud.dropna(inplace=True)
df_euraud.reset_index(drop=True, inplace=True)

# Preprocess Data
input_features = ["Close", "RSI", "MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist", "ATR"]
df_scaler = MinMaxScaler()
df_euraud[input_features] = df_scaler.fit_transform(df_euraud[input_features])

close_scaler = MinMaxScaler()
df_euraud['Close'] = close_scaler.fit_transform(df_euraud[['Close']])

# Split Data
train_size = int(len(df_euraud) * 0.8)
train_data = df_euraud[:train_size]
test_data = df_euraud[train_size:]

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length - 1):
        sequence = data[i : i + seq_length][input_features].values
        label = data["Close"].iloc[i + seq_length]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

# Build Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer="adam", loss="mean_squared_error")

# Train Model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Make Predictions
predictions = model.predict(test_x)
predictions = close_scaler.inverse_transform(predictions)

# Evaluate Model
mse = mean_squared_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
mae = mean_absolute_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Predict the next 6 days
next_6_days = [test_x[-1]]
for _ in range(6):
    predicted_value = model.predict(np.array([next_6_days[-1]]))[0][0]
    last_day_features = next_6_days[-1][-1].copy()
    last_day_features[0] = predicted_value
    new_day = np.concatenate((next_6_days[-1][1:], [last_day_features]), axis=0)
    next_6_days.append(new_day)

predicted_closes = [close_scaler.inverse_transform(np.array([[day[-1, 0]]]))[0][0] for day in next_6_days][1:]

# Loop for 5 times to get decisions for the 6 days, since there are 5 comparisons to make
decisions = ['Buy' if predicted_closes[i + 1] > predicted_closes[i] else 'Sell' if predicted_closes[i + 1] < predicted_closes[i] else 'Hold' for i in range(5)]

print(f"Predictions for the Next 6 Days: {predicted_closes}")
print(f"Predicted Action for the Next 5 Days based on the 6 Days predictions: {decisions}")


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
87/87 [==============================] - 10s 77ms/step - loss: 0.0075
Epoch 2/10
87/87 [==============================] - 5s 57ms/step - loss: 5.7513e-04
Epoch 3/10
87/87 [==============================] - 6s 68ms/step - loss: 4.9855e-04
Epoch 4/10
87/87 [==============================] - 6s 66ms/step - loss: 4.6233e-04
Epoch 5/10
87/87 [==============================] - 5s 57ms/step - loss: 4.3451e-04
Epoch 6/10
87/87 [==============================] - 7s 77ms/step - loss: 4.6371e-04
Epoch 7/10
87/87 [==============================] - 5s 57ms/step - loss: 4.0709e-04
Epoch 8/10
87/87 [==============================] - 6s 66ms/step - loss: 3.7048e-04
Epoch 9/10
87/87 [==============================] - 6s 68ms/step - loss: 3.5929e-04
Epoch 10/10
21/21 [==============================] - 1s 17ms/step
Mean Squared Error: 0.0002743589751456372
Mean Absolute Error: 0.012472223291569693
1/1 [=======================